In [1]:
import os
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "1"

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from ase.build import molecule
from ase.visualize import view
atoms = molecule('H2O')
view(atoms, viewer="x3d")

In [13]:
atoms.set_cell(20. * np.identity(3))
atoms.set_pbc((True, True, True))

In [19]:
from ase.calculators.espresso import Espresso

pseudopotentials = {
    "H":"H.pbe-rrkjus.UPF",
    "O":"O.pbe-rrkjus.UPF"
}

input_data = {
    'system': {
        'ecutwfc': 30,
        'ecutrho': 300,
    },
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos',
        'prefix': 'H2O_vacuum'
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta':0.4
    }
} 

calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

atoms.calc = calc

In [20]:
energy = atoms.get_potential_energy()
print(f"E = {energy:.3f} eV")

In [17]:
energy/13.605662285137

-34.26827917270261

The following cell uses bash to generate a new file in the working folder. This file, named environ.in, is the input used by Environ to setup the environment conditions.

In [3]:
%%bash
  cat > environ.in << EOF
&ENVIRON
   !
   verbose = 0
   environ_thr = 0.1
   environ_type = 'vacuum'
   env_electrostatic = .true.
   !
/
&BOUNDARY
/
&ELECTROSTATIC
   !
   pbc_correction = 'parabolic'
   pbc_dim = 0
   !
   tol = 1.d-9
   !
/
EOF